In [4]:
import pyscreenshot as ImageGrab
import keras

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is enabled with initial size: 65.0% of memory, cuDNN not available)


### model

In [15]:
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from itertools import chain
from keras import backend as K
from keras.layers import Dense, Activation, Convolution2D, Convolution3D, MaxPooling2D, MaxPooling3D, BatchNormalization, Flatten, ZeroPadding2D, Dropout
from keras.models import load_model, Sequential
from keras.optimizers import Adam

NB_FRAMES = 5
INPUT_SHAPE = (NB_FRAMES, 91 * 5, 118 * 5)
# OUTPUT_SHAPE = (1, OUTPUT_ROW, OUTPUT_COL)
OBJECTIVE_FUNCTION = 'binary_crossentropy'
METRICS = ['binary_crossentropy', 'accuracy']

def center_normalize(x):
    """Custom activation for online sample-wise center and std. normalization."""
    return (x - K.mean(x)) / K.std(x)

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(Activation(activation=center_normalize, input_shape=INPUT_SHAPE))
    model.add(ZeroPadding2D((1,1), input_shape=INPUT_SHAPE))
    # model.add(ZeroPadding2D((1,1),input_shape=(2, INPUT_ROW, INPUT_COL)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(9, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-5), loss=OBJECTIVE_FUNCTION, metrics=METRICS)

    return model

model = VGG_16()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
activation_4 (Activation)        (None, 5, 455, 590)   0           activation_input_4[0][0]         
____________________________________________________________________________________________________
zeropadding2d_40 (ZeroPadding2D) (None, 5, 457, 592)   0           activation_4[0][0]               
____________________________________________________________________________________________________
convolution2d_40 (Convolution2D) (None, 64, 455, 590)  2944        zeropadding2d_40[0][0]           
____________________________________________________________________________________________________
zeropadding2d_41 (ZeroPadding2D) (None, 64, 457, 592)  0           convolution2d_40[0][0]           
___________________________________________________________________________________________

### get screenshot

In [70]:
import numpy as np
import pyscreenshot as ImageGrab
im=ImageGrab.grab(bbox=(120,75,710,530)) # X1,Y1,X2,Y2
# im.show()
# im.save("fim.png")
pix = np.array(im)
print(type(pix))
print(pix.shape)
print(pix.mean(), pix.std())

<class 'numpy.ndarray'>
(455, 590, 3)
58.0387818961 67.4804627555


In [73]:
import numpy as np
import pyscreenshot as ImageGrab
im=ImageGrab.grab(bbox=(120,75,710,530)) # X1,Y1,X2,Y2
# im.show()
# im.save("errou.png")
pix2 = np.array(im)
print(type(pix))
print(pix.shape)
print(pix.mean(), pix.std())

<class 'numpy.ndarray'>
(455, 590, 3)
58.0387818961 67.4804627555


In [75]:
(pix2 - pix).mean()

1.3734488110759298

### joystick

In [13]:
with open('capture.out', 'rb') as f:
    my_list = pickle.load(f)
    print(my_list)

FileNotFoundError: [Errno 2] No such file or directory: 'capture.out'

## this is q-learning or a trial

In [78]:
from time import sleep
from qlearning4k.games.game import Game
from pymouse import PyMouse
from pykeyboard import PyKeyboard
import pyscreenshot as ImageGrab
import numpy as np
from matplotlib import pyplot as plt

k = PyKeyboard()

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

class Mario(Game):
    SLEEP = 0.1
    ACTIONS = {
        0: 'w',
        1: 's',
        2: 'a',
        3: 'd',
        4: 'f',
        5: 'g',
        6: 'r',
        7: 't',
        8: '\\',
    }
    
    def __init__(self, shape):
        self.reset()
        self.state = np.zeros(shape)
        self.update_state()
        k.press_key(self.ACTIONS[0])
        # which you then follow with a release of the key
        sleep(self.SLEEP)
        k.release_key(self.ACTIONS[0])
        self.__score = 0

    @property
    def name(self):
        return "Mario"

    @property
    def nb_actions(self):
        return len(self.ACTIONS.values())

    def reset(self):
        k.tap_key(k.function_keys[7])
        
    def update_state(self):
        im = ImageGrab.grab(bbox=(120, 75, 710, 530))
        im = im.resize((INPUT_SHAPE[2], INPUT_SHAPE[1]))
        self.last_state = self.state
        self.state = np.array(im)
        self.state = rgb2gray(self.state)

    def play(self, action):
#         print('this is action: ', action, self.ACTIONS[action])
        k.press_key(self.ACTIONS[action])
        sleep(self.SLEEP)
        k.release_key(self.ACTIONS[action])
        
        self.update_state()
        if (self.state - self.last_state).mean() > 1.0:
            self.__score += 1

    def get_state(self):
        return self.state

    def get_score(self):
        if self.is_over():
            return 0
        return self.__score
        
    def is_over(self):
        return self.state.mean() <= 1.0

    def is_won(self):
        return self.state.mean() > 1.0 and self.state.mean() < 30.0 and self.state.std() < 50.0

In [82]:
import sys
import os
sys.path.append(os.path.abspath('.'))
from agent import Agent
from time import sleep

sleep(3)
print('Start!')

mario = Mario((INPUT_SHAPE[1], INPUT_SHAPE[2]))
agent = Agent(model=model, nb_frames=NB_FRAMES)
agent.train(mario, batch_size=50, nb_epoch=1000, epsilon=.1)
# agent.play(mario)

Start!
score  0 0.0149866641833
score  0 -0.00520765505681
score  0 0.0106279344384
score  0 -0.00896466753585
score  0 -0.00420919724343
score  0 0.0124888768858
score  0 -0.0224818290184
score  0 0.0172806854163
score  0 0.00374420562488
score  0 -0.00783254982306
score  0 0.00842110635128
score  0 -0.000909912460421
score  0 -0.00229294840752
score  0 -0.00835342521885
score  0 -0.00520538647793
score  0 0.00641543676662
score  0 -0.00059931830881
score  0 0.0124790985286
score  0 0.00170498416837
score  0 0.00374961072825
score  0 0.0
score  0 -0.0251185993667
score  0 0.0257006369901
score  0 -0.00511250884709
score  0 -0.00229843546284
score  0 -0.0574892568449
score  0 0.0561604246601
score  0 0.00220619854722
score  0 -0.00475439746694
score  0 -0.0658020003725
score  0 0.0743547513503
score  0 -0.0535401788042
score  0 0.0653223952319
score  0 -0.00420919724343
score  0 -0.00783791395046
score  0 0.00841995157385
score  0 -0.000903311603651
score  0 -0.0680342559136
score  0 0

MissingInputError: ("An input of the graph, used to compute DimShuffle{x,x}(keras_learning_phase), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.", keras_learning_phase)

In [32]:
sleep(3)

k.press_key('w')
# which you then follow with a release of the key
sleep(0.1)
k.release_key('w')
print ('vaca')

vaca
